In [4]:
from google.colab import drive
drive.mount('/content/drive')
!pip install tensorly
!pip install pmdarima
%cd /content/drive/My Drive/Colab Notebooks/Group30_Final_Project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content


In [0]:
from input.M4DataSet.transformed_data.transform_m4_year import m4_year

In [0]:
import numpy as np
m4_year = m4_year[np.random.randint(0, (m4_year.shape[0] -1 ), 
                                              size=(m4_year.shape[0] // 100),
                                              dtype='l')]

In [55]:
m4_year.shape

(230, 13)

In [0]:
import numpy as np

from BHT_ARIMA import BHTARIMA
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_BHTARIMA(data):


      p = 3 # p-order
      d = 2 # d-order
      q = 1 # q-order
      taus = [230,5] # MDT-rank
      Rs = [5,5] # tucker decomposition ranks
      k =  15 # iterations
      tol = 0.001 # stop criterion
      Us_mode = 3 # orthogonality mode
      testsize = 0.1

      T = data.shape[-1]
      T_test = int((T * testsize) // 1)
      result_full = np.zeros([data.shape[0], T_test])

      n_round = 0

      for i in range(T_test):

        ts = data[..., i:T-T_test+i].copy()
        n_round += 1
        model = BHTARIMA(ts, p, d, q, taus, Rs, k, tol, verbose=0, Us_mode=Us_mode)
        result, _ = model.run()
        result_full[..., i] = result[..., -1]

      label = data[..., -T_test:]

      stat = {}
      stat['acc'] = get_acc(result_full, label)
      stat['nrmse'] = nrmse(result_full, label)

      return(stat)

In [25]:
BHTARIMA_m4_year = run_BHTARIMA(m4_year)
BHTARIMA_m4_year

{'acc': 0.9314507516526322, 'nrmse': 0.15602034392665579}

In [0]:
import numpy as np
import pmdarima as pm
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_ARIMA(data):

      testsize = 0.1

      T = data.shape[-1]
      T_test = int((T * testsize) // 1)
      result_full = np.zeros([data.shape[0], T_test])

      total_time = 0
      n_round = 0

      for i in range(T_test):

        y = data[..., i:T-T_test+i].copy()
        n_round += 1

        for j in range(y.shape[0]):
          
          model = pm.AutoARIMA(seasonal=True, m= 1,suppress_warnings=True,trace=False,error_action="ignore")

          result = model.fit_predict(y[j], n_periods=1)
          result_full[j, i] = result[..., -1]


      true_value = data[..., -T_test:]


      stat = {}
      stat['acc'] = get_acc(result_full, true_value)
      stat['nrmse'] = nrmse(result_full, true_value)


      return(stat)

In [30]:
ARIMA_m4_year = run_ARIMA(m4_year)
ARIMA_m4_year

{'acc': 0.9283621508084372, 'nrmse': 0.2049158083504426}

In [0]:
import numpy as np
import pandas as pd

from fbprophet import Prophet
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_Prophet(data):


      testsize = 0.1
      start = '2017-3'
      freq = 'D'


      T = data.shape[-1]
      T_test = int((T * testsize) // 1)
      result_full = np.zeros([data.shape[0], T_test])

      n_round = 0

      time_st = pd.period_range(start=start, periods=T, freq=freq).to_timestamp()


      for i in range(T_test):

        y = data[..., i:T-T_test+i+1].copy()
        n_round += 1
        ds = time_st[i:T-T_test+i+1].copy()
        


        for j in range(y.shape[0]):


          DF_ts = pd.DataFrame()
          DF_ts['ds'] = ds.copy()
          DF_ts['y'] = y[j].copy()



          model = Prophet(yearly_seasonality=True,daily_seasonality=True,weekly_seasonality=True,n_changepoints=8)
          model.fit(DF_ts[i:T-T_test+i])

          result = model.predict(DF_ts[['ds']].iloc[[-1]])['yhat'][0]
          result_full[j, i] = result


      true_value = data[..., -T_test:]


      stat = {}
      stat['acc'] = get_acc(result_full, true_value)
      stat['nrmse'] = nrmse(result_full, true_value)


      return(stat)
 

In [61]:
Prophet_m4_year = run_Prophet(m4_year)
Prophet_m4_year

{'acc': 0.8598976809903849, 'nrmse': 0.3280863781196177}

In [0]:
import numpy as np

from statsmodels.tsa.api import VAR
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_VAR(data):
  
      testsize = 0.1
      p = 1

      T = data.shape[-1]
      T_test = int((T * testsize) // 1)
      result_full = np.zeros([data.shape[0], T_test])

      total_time = 0
      n_round = 0

      for i in range(T_test):

        ts = (data[..., i:T-T_test+i].copy())
        n_round += 1
        model = VAR(ts)
        result = model.fit(p).forecast(ts, 1)
        result_full[..., i] = result[..., -1]

      label = data[..., -T_test:]

      stat = {}
      stat['acc'] = get_acc(result_full, label)
      stat['nrmse'] = nrmse(result_full, label)

      return(stat)

In [71]:
VAR_m4_year = run_VAR(m4_year)
VAR_m4_year


{'acc': 0.5038034911470338, 'nrmse': 0.8430721205741382}

In [0]:
import numpy as np

import xgboost as xgb
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_XGB(data, test_size):

      T_test = test_size

      result_full = np.zeros([data.shape[0], T_test])
      parameters = {'max_depth':(3,4,5,6,7,8,9,10), 
              'subsample':[0.5,0.6, 0.7, 0.8, 0.9],
               'lambda' :[0, 0.25, 0.5, 0.75, 1]}
      clf = GridSearchCV(XGBRegressor(objective ='reg:squarederror'), parameters, verbose = False, cv = 3)
      X = data[..., :-T_test-1]
      y = data[..., -T_test-1]
      clf.fit(X, y)
      for i in range(T_test):
        X_test = data[..., i+1:-T_test+i]
        result = clf.predict(X_test)
        result_full[..., i] = result

      label = data[..., -T_test:]

      stat = {}
      stat['acc'] = get_acc(result_full, label)
      stat['nrmse'] = nrmse(result_full, label)

      return(stat)

In [75]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
import warnings

warnings.filterwarnings("ignore")

XGB_m4_year = run_XGB(m4_year, test_size=m4_year.shape[1] // 10)
XGB_m4_year

{'acc': 0.9222093293861576, 'nrmse': 0.1423833876401753}